In [49]:
import glob
import numpy as np
import pandas as pd
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
from collections import Counter

In [2]:
PATH = '../lab1/ustawy/'
file_names = sorted(glob.glob(PATH + "*"))
file_names[:5]

['../lab1/ustawy/1993_599.txt',
 '../lab1/ustawy/1993_602.txt',
 '../lab1/ustawy/1993_645.txt',
 '../lab1/ustawy/1993_646.txt',
 '../lab1/ustawy/1994_150.txt']

In [3]:
law_acts =[]
for file_name in file_names:
    with open(file_name, "r", encoding="utf-8") as f:
        law_acts.append(f.read().lower())

# Zadanie 1

Tokenizacja każdego projektu ustawy przy użyciu tokenizera spacy

In [4]:
lang = Polish()
tokenizer = Tokenizer(lang.vocab)

In [5]:
%%time
tokens_per_act = []

for law_act in law_acts:
    tokens_per_act.append(tokenizer(law_act))

CPU times: user 37.1 s, sys: 560 ms, total: 37.6 s
Wall time: 38 s


# Zadanie 2

Obliczenie liczby wystąpień bigramów

In [7]:
%%time
bigrams_counter = {}
for tokens in tokens_per_act:
    token_strings = [token.string.strip().lower() for token in tokens]
    bigrams = zip(token_strings[:-1], token_strings[1:])
    for bigram in bigrams:
        bigrams_counter[bigram] = bigrams_counter.get(bigram, 0) + 1

CPU times: user 12.8 s, sys: 200 ms, total: 13 s
Wall time: 13.1 s


In [14]:
list(bigrams_counter.items())[:10]

[(('', 'dz.u.'), 950),
 (('dz.u.', 'z'), 1212),
 (('z', '1993'), 512),
 (('1993', 'r.'), 763),
 (('r.', 'nr'), 17006),
 (('nr', '129,'), 121),
 (('129,', 'poz.'), 114),
 (('poz.', '599'), 1),
 (('599', ''), 1),
 (('', 'ustawa'), 2383)]

# Zadanie 3

Odfiltrowanie bigramów zawierających nie litery

In [18]:
%%time
new_bigrams_counter = {}
for bigram, count in bigrams_counter.items():
    if bigram[0].isalpha() and bigram[1].isalpha():
        new_bigrams_counter[bigram] = count
bigrams_counter = new_bigrams_counter

CPU times: user 484 ms, sys: 15.9 ms, total: 500 ms
Wall time: 500 ms


In [152]:
list(bigrams_counter.items())[:10]

[(('z', 'dnia'), 9023),
 (('o', 'zmianie'), 1175),
 (('zmianie', 'ustawy'), 829),
 (('ustawy', 'o'), 1412),
 (('o', 'podatku'), 496),
 (('podatku', 'od'), 362),
 (('od', 'towarów'), 248),
 (('towarów', 'i'), 505),
 (('i', 'usług'), 359),
 (('oraz', 'o'), 897)]

# Zadanie 4

Obliczenie PMI dla każdego bigramu. W tym celu potrzebuje obliczyć prawdopodobieństwa związane z pojedynczymi słowami i bigramami, stąd wyliczanie licznika tokenów i całkowitych liczb tokenów i bigramów

In [26]:
%%time
# obliczenie słownika częstotliwości występowania tokenów
tokens_counter = {}
for tokens in tokens_per_act:
    for token in tokens:
        token_string = token.string.strip().lower()
        tokens_counter[token_string] = tokens_counter.get(token_string, 0) + 1
# usuniecie tokenow zawierajacych nie litery
new_tokens_counter = {}
for token, count in tokens_counter.items():
    if token.isalpha():
        new_tokens_counter[token] = count
tokens_counter = new_tokens_counter
# obliczenie całkowitej liczby tokenow
total_number_of_tokens = sum(list(tokens_counter.values()))

CPU times: user 11.7 s, sys: 11.5 ms, total: 11.7 s
Wall time: 11.8 s


In [27]:
print(list(tokens_frequency_list.items())[:10])
total_number_of_tokens

[('z', 82177), ('nr', 45125), ('ustawa', 3185), ('dnia', 17711), ('grudnia', 2107), ('o', 64702), ('zmianie', 1471), ('ustawy', 9677), ('podatku', 1813), ('od', 16690)]


2950389

In [29]:
# obliczenie całkowitej liczby bigramow
total_number_of_bigrams = sum(list(bigrams_counter.values()))
total_number_of_bigrams

2110524

In [34]:
%%time
# obliczenie PMI
bigrams_pmi = {}
for bigram, count in bigrams_counter.items():
    p_bigram = count / total_number_of_bigrams
    p_x = tokens_counter[bigram[0]] / total_number_of_tokens
    p_y = tokens_counter[bigram[1]] / total_number_of_tokens
    pmi = np.log(p_bigram / (p_x * p_y))
    bigrams_pmi[bigram] = pmi

CPU times: user 1.57 s, sys: 24.1 ms, total: 1.59 s
Wall time: 1.6 s


In [36]:
list(bigrams_pmi.items())[:10]

[(('z', 'dnia'), 3.2414085732051903),
 (('o', 'zmianie'), 3.930226666270485),
 (('zmianie', 'ustawy'), 5.481463571858889),
 (('ustawy', 'o'), 2.230156163465908),
 (('o', 'podatku'), 2.8587386762637395),
 (('podatku', 'od'), 3.898789336621013),
 (('od', 'towarów'), 3.6876780997878735),
 (('towarów', 'i'), 2.7211122402413706),
 (('i', 'usług'), 2.6121881188553786),
 (('oraz', 'o'), 0.5441337113232051)]

# Zadanie 5

Wyświetlenie 10 bigramów z największym PMI

In [39]:
top_pmi_bigrams = sorted(list(bigrams_pmi.items()), key=(lambda x: (-x[1], x[0])))[:10]
top_pmi_bigrams

[(('administracjami', 'drogowymi'), 15.232448352205974),
 (('aegroti', 'suprema'), 15.232448352205974),
 (('aerodynamicznej', 'szorstkości'), 15.232448352205974),
 (('agregatach', 'pralniczych'), 15.232448352205974),
 (('agricoltura', 'biologica'), 15.232448352205974),
 (('agriculture', 'biologique'), 15.232448352205974),
 (('agrotechniki', 'nienaruszającej'), 15.232448352205974),
 (('aktowi', 'normatywnemu'), 15.232448352205974),
 (('aleksander', 'małachowski'), 15.232448352205974),
 (('aleksandrów', 'kujawski'), 15.232448352205974)]

# Zadanie 6

Wyświetlenie 10 bigramów z największym PMI, które występują przynajmniej 5 razy

In [44]:
%%time
# obliczenie PMI dla bigramów występujących przynajmniej 5 razy
common_bigrams_pmi = {}
for bigram, count in bigrams_counter.items():
    if count < 5:
        continue
    p_bigram = count / total_number_of_bigrams
    p_x = tokens_counter[bigram[0]] / total_number_of_tokens
    p_y = tokens_counter[bigram[1]] / total_number_of_tokens
    pmi = np.log(p_bigram / (p_x * p_y))
    common_bigrams_pmi[bigram] = pmi

CPU times: user 316 ms, sys: 0 ns, total: 316 ms
Wall time: 319 ms


In [45]:
top_common_pmi_bigrams = sorted(list(common_bigrams_pmi.items()), key=(lambda x: (-x[1], x[0])))[:10]
top_common_pmi_bigrams

[(('grzegorz', 'schetyna'), 13.623010439771873),
 (('młyny', 'kulowe'), 13.623010439771873),
 (('ręcznego', 'miotacza'), 13.623010439771873),
 (('zaszkodzić', 'wynikom'), 13.623010439771873),
 (('świeckie', 'przygotowujące'), 13.623010439771873),
 (('adama', 'mickiewicza'), 13.440688882977918),
 (('mleczka', 'makowego'), 13.440688882977918),
 (('przeponowe', 'rurowe'), 13.440688882977918),
 (('chwytów', 'obezwładniających'), 13.286538203150661),
 (('lambrekiny', 'okienne'), 13.286538203150661)]

# Zadanie 7

Obliczenie LLR dla bigramów. W tym celu będę potrzebował obliczyć liczby bigramów zaczynających się na a i nie kończących na b i kończących się na b i nie zaczynających na a, stąd obliczenia liczb bigramów zaczynających się na a i bigramów kończących się na b

In [68]:
# compute counter of bigrams starting with a
bigrams_starting_with_a_counter = {}
for bigram, count in bigrams_counter.items():
    a = bigram[0]
    bigrams_starting_with_a_counter[a] = bigrams_starting_with_a_counter.get(a, 0) + count
    
# compute counter of bigrams ending with b
bigrams_ending_with_b_counter = {}
for bigram, count in bigrams_counter.items():
    b = bigram[1]
    bigrams_ending_with_b_counter[b] = bigrams_ending_with_b_counter.get(b, 0) + count

In [79]:
def H(k):
    N = np.sum(k)
    return np.sum(k/N * np.log(k/N + (k==0)))

przy obliczeniu neither_a_nor_b poniżej, dodaje a_and_b, bo odjąłem je dwukrotnie odejmując bigramy zaczynające się na a i bigramy kończące się na b

In [87]:
%%time
bigrams_llr = {}
for bigram, count in bigrams_counter.items():
    a, b = bigram
    a_and_b = count
    a_without_b = bigrams_starting_with_a_counter[a] - a_and_b
    b_but_not_a = bigrams_ending_with_b_counter[b] - a_and_b
    neither_a_nor_b = total_number_of_bigrams - bigrams_starting_with_a_counter[a] \
        - bigrams_ending_with_b_counter[b] + a_and_b
    
    k = np.array([[a_and_b, b_but_not_a], [a_without_b, neither_a_nor_b]])
    row_sums = np.array([a_and_b + b_but_not_a, a_without_b + neither_a_nor_b])
    col_sums = np.array([a_and_b + a_without_b, b_but_not_a + neither_a_nor_b])
    llr = 2 * np.sum(k) * (H(k) - H(row_sums) - H(col_sums))
    bigrams_llr[bigram] = llr

CPU times: user 1min 2s, sys: 111 ms, total: 1min 3s
Wall time: 1min 3s


In [88]:
list(bigrams_llr.items())[:10]

[(('z', 'dnia'), 47165.490621818884),
 (('o', 'zmianie'), 7695.115800574815),
 (('zmianie', 'ustawy'), 8025.075607216169),
 (('ustawy', 'o'), 5188.066217788072),
 (('o', 'podatku'), 1676.5302203803762),
 (('podatku', 'od'), 2048.566391893052),
 (('od', 'towarów'), 1243.962431029594),
 (('towarów', 'i'), 1897.375241420914),
 (('i', 'usług'), 1248.9969723116897),
 (('oraz', 'o'), 716.9105188653832)]

# Zadanie 8

Wyświetlenie 10 bigramów z największym llr

In [89]:
top_llr_bigrams = sorted(list(bigrams_llr.items()), key=(lambda x: (-x[1], x[0])))[:10]
top_llr_bigrams

[(('mowa', 'w'), 162056.18578064922),
 (('których', 'mowa'), 103860.96812895118),
 (('o', 'których'), 81588.75159791514),
 (('którym', 'mowa'), 67847.08382895816),
 (('dodaje', 'się'), 62515.89447212826),
 (('do', 'spraw'), 57437.50713390384),
 (('o', 'którym'), 53202.37664633561),
 (('i', 'nr'), 51618.5219313482),
 (('z', 'dnia'), 47165.490621818884),
 (('na', 'podstawie'), 45750.29767443445)]

# Zadanie 9

Obliczenie liczb wystąpień trigramów i odfiltrowanie tych, które zawierają nie litery

In [92]:
%%time
trigrams_counter = {}
for tokens in tokens_per_act:
    token_strings = [token.string.strip().lower() for token in tokens]
    trigrams = zip(token_strings[:-2], token_strings[1:-1], token_strings[2:])
    for trigram in trigrams:
        trigrams_counter[trigram] = trigrams_counter.get(trigram, 0) + 1
        
new_trigrams_counter = {}
for trigram, count in trigrams_counter.items():
    if trigram[0].isalpha() and trigram[1].isalpha() and trigram[2].isalpha():
        new_trigrams_counter[trigram] = count
trigrams_counter = new_trigrams_counter

list(trigrams_counter.items())[:10]

CPU times: user 13.4 s, sys: 216 ms, total: 13.6 s
Wall time: 13.7 s


[(('o', 'zmianie', 'ustawy'), 750),
 (('zmianie', 'ustawy', 'o'), 614),
 (('ustawy', 'o', 'podatku'), 123),
 (('o', 'podatku', 'od'), 106),
 (('podatku', 'od', 'towarów'), 194),
 (('od', 'towarów', 'i'), 223),
 (('towarów', 'i', 'usług'), 277),
 (('oraz', 'o', 'podatku'), 46),
 (('o', 'podatku', 'akcyzowym'), 34),
 (('w', 'ustawie', 'z'), 3359)]

# Zadanie 10

Obliczenie PMI i LLR dra trigramów (a przynajmniej tego co wydaje mi się, że powinno być analogiczną miarą)

## PMI

Obliczam je analogicznie do PMI dla bigramów, tylko w mianowniku dodaje prawdopodobieństwo wystąpienia trzeciego słowa. Po drodze muszę zliczyć łączną liczbę trigramów, wartości do obliczenia prawdopodobieństw związanych z pojedynczymi słowami już mam policzone wcześniej.

In [94]:
# obliczenie całkowitej liczby trigramów
total_number_of_trigrams = sum(list(trigrams_counter.values()))
total_number_of_trigrams

1534380

In [95]:
%%time
# obliczenie PMI dla trigramów występujących przynajmniej 5 razy
common_trigrams_pmi = {}
for trigram, count in trigrams_counter.items():
    if count < 5:
        continue
    p_trigram = count / total_number_of_trigrams
    p_x = tokens_counter[trigram[0]] / total_number_of_tokens
    p_y = tokens_counter[trigram[1]] / total_number_of_tokens
    p_z = tokens_counter[trigram[2]] / total_number_of_tokens
    pmi = np.log(p_trigram / (p_x * p_y * p_z))
    common_trigrams_pmi[trigram] = pmi

CPU times: user 304 ms, sys: 0 ns, total: 304 ms
Wall time: 321 ms


### Wyświetlenie 10 trigramów z największym PMI

In [97]:
top_common_pmi_trigrams = sorted(list(common_trigrams_pmi.items()), key=(lambda x: (-x[1], x[0])))[:10]
top_common_pmi_trigrams

[(('piłce', 'nożnej', 'uefa'), 25.219381308411705),
 (('profilem', 'zaufanym', 'epuap'), 25.20102405289333),
 (('religijne', 'uroczystości', 'pogrzebowe'), 25.160438772778253),
 (('finałowego', 'turnieju', 'mistrzostw'), 25.142089634110057),
 (('potwierdzonym', 'profilem', 'zaufanym'), 24.883489017291872),
 (('turnieju', 'mistrzostw', 'europy'), 24.879725369642568),
 (('grożącą', 'jemu', 'samemu'), 24.27742282978897),
 (('bankowemu', 'funduszowi', 'gwarancyjnemu'), 24.21038117843328),
 (('komunalne', 'osady', 'ściekowe'), 24.179609519766526),
 (('konfesyjne', 'nauczanie', 'religii'), 24.157136663914468)]

## LLR

Obliczam je w miarę analogicznie do LLR dla bigramów, tylko aby policzyć macierz k analogiczną do confusion matrix dla bigramów, potrzebuję obliczyć 8 wartości pokrywających wszystkie kombinacje wystąpień i braków wystąpień kolejnych słów w trigramach. Ostatecznie dostanę więc tablicę 2x2x2 i przy obliczaniu uwzględnię sumy tej tablicy w każdej z 3 osi (analogicznie jak wcześniej 2). Żeby to wszystko policzyć muszę przygotować liczby trigramów zawierających konkretne słowo na konretnej pozycji oraz liczby trigramów zawierających konkretne pary słów na konkretnych pozycjach.

In [103]:
# compute counter of trigrams starting with a
trigrams_with_a_counter = {}
for trigram, count in trigrams_counter.items():
    a = trigram[0]
    trigrams_with_a_counter[a] = trigrams_with_a_counter.get(a, 0) + count
    
# compute counter of trigrams with b on 2nd position
trigrams_with_b_counter = {}
for trigram, count in trigrams_counter.items():
    b = trigram[1]
    trigrams_with_b_counter[b] = trigrams_with_b_counter.get(b, 0) + count
    
# compute counter of trigrams ending with c
trigrams_with_c_counter = {}
for trigram, count in trigrams_counter.items():
    c = trigram[2]
    trigrams_with_c_counter[c] = trigrams_with_c_counter.get(c, 0) + count

In [101]:
# compute counter of trigrams starting with a and b
trigrams_with_a_b_counter = {}
for trigram, count in trigrams_counter.items():
    ab = (trigram[0], trigram[1])
    trigrams_with_a_b_counter[ab] = trigrams_with_a_b_counter.get(ab, 0) + count
    
# compute counter of trigrams starting with a and ending with c
trigrams_with_a_c_counter = {}
for trigram, count in trigrams_counter.items():
    ac = (trigram[0], trigram[2])
    trigrams_with_a_c_counter[ac] = trigrams_with_a_c_counter.get(ac, 0) + count
    
# compute counter of trigrams with b in the middle and ending with c
trigrams_with_b_c_counter = {}
for trigram, count in trigrams_counter.items():
    bc = (trigram[1], trigram[2])
    trigrams_with_b_c_counter[bc] = trigrams_with_b_c_counter.get(bc, 0) + count

In [102]:
def H(k):
    N = np.sum(k)
    return np.sum(k/N * np.log(k/N + (k==0)))

In [116]:
%%time
trigrams_llr = {}
for trigram, count in trigrams_counter.items():
    a, b, c = trigram
    a1_b1_c1 = count
    a1_b1_c0 = trigrams_with_a_b_counter[(a, b)] - a1_b1_c1
    a0_b1_c1 = trigrams_with_b_c_counter[(b, c)] - a1_b1_c1
    a1_b0_c1 = trigrams_with_a_c_counter[(a, c)] - a1_b1_c1
    # poniżej muszę dodać a1_b1_c1 bo odejmuję je dwukrotnie
    a1_b0_c0 = trigrams_with_a_counter[a] - trigrams_with_a_b_counter[(a, b)] - trigrams_with_a_c_counter[(a, c)] + a1_b1_c1
    a0_b1_c0 = trigrams_with_b_counter[b] - trigrams_with_a_b_counter[(a, b)] - trigrams_with_b_c_counter[(b, c)] + a1_b1_c1
    a0_b0_c1 = trigrams_with_c_counter[c] - trigrams_with_a_c_counter[(a, c)] - trigrams_with_b_c_counter[(b, c)] + a1_b1_c1
    # poniżej muszę pododawać liczby wystąpień par, bo odejmuję je dwukrotnie i odjąć a1_b1_c1 bo odejmuje je i dodaje tyle samo razy
    a0_b0_c0 = total_number_of_trigrams - trigrams_with_a_counter[a] - trigrams_with_b_counter[b] - trigrams_with_c_counter[c] \
        + trigrams_with_a_b_counter[(a, b)] + trigrams_with_b_c_counter[(b, c)] + trigrams_with_a_c_counter[(a, c)] \
        - a1_b1_c1
    
    k = np.array([[[a1_b1_c1, a1_b1_c0], 
                   [a1_b0_c1, a1_b0_c0]],
                  [[a0_b1_c1, a0_b1_c0],
                   [a0_b0_c1, a0_b0_c0]]])
    
    llr = 2 * np.sum(k) * (H(k) - H(k.sum(axis=0)) - H(k.sum(axis=1)) - H(k.sum(axis=2)))
    trigrams_llr[trigram] = llr

CPU times: user 2min 50s, sys: 485 ms, total: 2min 51s
Wall time: 2min 51s


In [119]:
top_llr_trigrams = sorted(list(trigrams_llr.items()), key=(lambda x: (-x[1], x[0])))[:10]
top_llr_trigrams

[(('w', 'których', 'w'), 1483096.226500892),
 (('w', 'którym', 'w'), 1444367.400738985),
 (('w', 'której', 'w'), 1401846.6412625448),
 (('w', 'i', 'i'), 1401717.2295618653),
 (('w', 'ustawie', 'w'), 1383349.626962069),
 (('w', 'tym', 'w'), 1382262.0407949397),
 (('w', 'zakresie', 'w'), 1380645.0902347357),
 (('w', 'pracy', 'w'), 1377009.420790753),
 (('w', 'określonych', 'w'), 1375598.9852127223),
 (('w', 'życie', 'w'), 1370346.52857257)]

# Zadanie 11

Utworzenie tabel porównujących otrzymane wyniki. Ze względu na to, że wartości poszczególnych miar mogą w ogóle nie mieć ze sobą nic wspólnego tworzę dataframe'y pokazujące indeks na którym znalazły się konkretne bigramy w listach posortowanych po malejącej wartości danych metryk. Czasami dla wystąpień w liście bigramów posortowanych po PMI, które występują co najmniej 5 razy jest inf, jeśli bigramy nie występują 5 razy. Analogicznie przygotowałem dataframe dla trigramów.

## Bigrams

In [131]:
# będę badał bigramy, które były najwyżej oceniany przez każdą z miar
top_bigrams = []
for bigram, score in top_llr_bigrams + top_pmi_bigrams + top_common_pmi_bigrams:
    top_bigrams.append(bigram)

In [133]:
# tworzę posortowane listy bigramów zgodnie z różnymi miarami
s_bigrams_llr = [b for b, _ in sorted(list(bigrams_llr.items()), key=(lambda x: (-x[1], x[0])))]
s_common_bigrams_pmi = [b for b, _ in sorted(list(common_bigrams_pmi.items()), key=(lambda x: (-x[1], x[0])))]
s_bigrams_pmi = [b for b, _ in sorted(list(bigrams_pmi.items()), key=(lambda x: (-x[1], x[0])))]

In [146]:
# wyliczam indeksy bigramów w każdej z list
llr_place = []
common_pmi_place = []
pmi_place = []
for bigram in top_bigrams:
    llr_place.append(s_bigrams_llr.index(bigram) + 1)
    try:
        common_pmi_place.append(s_common_bigrams_pmi.index(bigram) + 1)
    except:
        common_pmi_place.append(np.inf)
    pmi_place.append(s_bigrams_pmi.index(bigram) + 1)

In [147]:
pd.DataFrame({'llr_index': llr_place, 
              'common_pmi_index': common_pmi_place, 
              'pmi_index': pmi_place}, 
             index=top_bigrams)

,llr_index,common_pmi_index,pmi_index
"(mowa, w)",1,37678.0,290211
"(których, mowa)",2,23857.0,191197
"(o, których)",3,30495.0,242352
"(którym, mowa)",4,23742.0,190139
"(dodaje, się)",5,24963.0,201473
"(do, spraw)",6,28454.0,226311
"(o, którym)",7,30237.0,239934
"(i, nr)",8,45712.0,338681
"(z, dnia)",9,35439.0,273479
"(na, podstawie)",10,26625.0,213367


## Trigrams

analogicznie jak dla bigramów

In [148]:
top_trigrams = []
for trigram, score in top_llr_trigrams + top_common_pmi_trigrams:
    top_trigrams.append(trigram)

In [149]:
s_trigrams_llr = [b for b, _ in sorted(list(trigrams_llr.items()), key=(lambda x: (-x[1], x[0])))]
s_common_trigrams_pmi = [b for b, _ in sorted(list(common_trigrams_pmi.items()), key=(lambda x: (-x[1], x[0])))]

In [150]:
llr_place = []
common_pmi_place = []
for trigram in top_trigrams:
    llr_place.append(s_trigrams_llr.index(trigram) + 1)
    try:
        common_pmi_place.append(s_common_trigrams_pmi.index(trigram) + 1)
    except:
        common_pmi_place.append(np.inf)

In [151]:
pd.DataFrame({'llr_index': llr_place, 
              'common_pmi_index': common_pmi_place}, 
             index=top_trigrams)

,llr_index,common_pmi_index
"(w, których, w)",1,38783.0
"(w, którym, w)",2,38784.0
"(w, której, w)",3,38779.0
"(w, i, i)",4,inf
"(w, ustawie, w)",5,38731.0
"(w, tym, w)",6,38480.0
"(w, zakresie, w)",7,38762.0
"(w, pracy, w)",8,inf
"(w, określonych, w)",9,inf
"(w, życie, w)",10,38694.0


# Zadanie 12 Wnioski

1. Why do we have to filter the bigrams, rather than the token sequence?

Odfiltrowanie błędnych tokenów i potem tworzenie z nich bigramów / trigramów doprowadziło by do uzyskiwania sztucznych multiwordów nie występujących na prawdę w tekscie. Przykładowo z fragmentu zdania 'z art. 5a skreśla się' otrzymalibyśmy błędny bigram (z, skreśla).

2. Which measure (PMI, PMI with filtering, LLR) works better for the bigrams and which for the trigrams?

Dla trigramów wyraźnie lepiej poradziła sobie miara PMI (po odfiltrowaniu trigramów występujących 5 razy), od miary LLR. Ponadto dobre frazy zwrócone przez PMI są bardzo nisko na liście fraz posortowanych po LLR.

Dla bigramów najgorzej poradziła sobie miara PMI bez odfiltrowania bigramów występujących często, bo pozwoliła uzyskać frazy występujące jednokrotnie, więc mało istotne. LLR i PMI po odfiltrowaniu zwróciły bardzo różne wyniki, ale ciężko stwierdzić, które z nich były lepsze. PMI zwróciło frazy rzeczywiście łączące sie często ze sobą jak 'adama mickiewicza' czy 'chwytów obezwładniających', ale nie występujące za często, a LLR frazy o wiele częściej łączące się ze sobą ('o których', 'z dnia'), ale mało ciekawe, związane z typowymi konstrukcjami językowymi.

3. What types of expressions are discovered by the methods.

LLR znajduje często występujące frazy łączące się ze sobą, ale będące zwykle utartymi konstrukcjami językowymi ('dodaje się', 'o których', 'w tym w'), czyli frazy które często się ze sobą łączą i występują często.
PMI, bez filtrowania fraz występujących często, zwraca frazy występujące bardzo rzadko i zawierające słowa występujące bardzo rzadko ('aerodynamicznej szorstkości', 'agregatach pralniczych').
PMI, z filtrowaniem fraz występujących często, zwraca frazy rzeczywiście łączące się często ze sobą, ale nie wsytępujące najczęściej w naszym zbiorze dokumentów. W związku z tym nie są to raczej konstrukcje językowe, tylko frazy, których pojedyczne słowa jak już wystąpią, to zazwyczaj wchodzą właśnie w skład tej frazy ('profilem zaufanym epuap', 'komunalne osady ściekowe', 'grzegorz schetyna')


4. Can you devise a different type of filtering that would yield better results?

Można pokombinować ze zwiększeniem warunku liczby wystąpień, aby uznać multiword za występujący często, przed liczeniem PMI. Można też pomyśleć o ważeniu miar LLR i PMI.
